In [1]:
import os
cachedir = '/jumbo/yaoqingyang/vipul/cache'
os.environ["CUDA_VISIBLE_DEVICES"] = "0"
os.environ["TRANSFORMERS_CACHE"] = cachedir
os.environ["HF_DATASETS_CACHE"]= cachedir
from model import get_model
from loader.layers import param_count
from loader.data_module import make_data_module
import json
import time
import torch
import random
import logging
import argparse
import numpy as np
import transformers
from pathlib import Path
import torch.backends.mps
import torch.backends.cudnn
from torch.cuda import (
    max_memory_allocated,
    reset_peak_memory_stats,
    reset_max_memory_allocated,
    memory_allocated,
)
from loader.logger import get_logger
from transformers import set_seed
# from accelerate import Accelerator
from torch.utils.data import DataLoader
from os.path import exists, join, isdir
from dataclasses import dataclass, field
from typing import Optional, Dict, Sequence

from transformers.utils.logging import (
    set_verbosity_error as transformers_vb_err,
)
from datasets.utils.logging import (
    set_verbosity_error as datasets_vb_err,
)
from transformers import Seq2SeqTrainer
from traineval.eval import eval_func
logger = logging.getLogger(__name__)
IGNORE_INDEX = -100
DEFAULT_PAD_TOKEN = "[PAD]"
from llamaft import ModelArguments, DataArguments, TrainingArguments, GenerationArguments

/jumbo/yaoqingyang/vipul/miniconda3/lib/python3.11/site-packages/transformers/utils/hub.py:124: FutureWarning: Using `TRANSFORMERS_CACHE` is deprecated and will be removed in v5 of Transformers. Use `HF_HOME` instead.
  warnings.warn(
Matplotlib created a temporary cache directory at /tmp/matplotlib-3dqbmtll because the default path (/thayerfs/home/f00752b/.config/matplotlib) is not a writable directory; it is highly recommended to set the MPLCONFIGDIR environment variable to a writable directory, in particular to speed up the import of Matplotlib and to better support multiprocessing.
INFO:datasets:PyTorch version 2.1.2 available.


In [2]:
# Setting up the arguments

model_args = ModelArguments(
    model_name_or_path="meta-llama/Llama-2-7b-hf"
)

data_args = DataArguments(
    eval_dataset_size=1024,
    max_eval_samples=1000,
    source_max_len = 1024,
    target_max_len = 256,
    dataset="alpaca", # DATASET [alpaca|chip2|self-instruct|hh-rlhf|oasst1|longform]
)

training_args = TrainingArguments(
    output_dir="./output",
    data_seed=7,
    save_strategy="steps",
    evaluation_strategy="steps",
    logging_strategy="steps",
    do_eval=True,
    eval_steps=187,
    adam_beta2=0.999,

    learning_rate=2e-6,     # LEARNING RATE
    
    max_steps=2000,         # NUMBER OF STEPS

    sortby="alpha",         # CAN DO "alpha" or "lora" or "dora"

    num_layers=12,           # NUMBER OF LAYERS FOR FULL FINE-TUNING

    per_device_train_batch_size = 2, # BATCH-SIZE
    memlog=False,
)

generation_args = GenerationArguments(
    max_new_tokens=256 # default is 256
)

# If you need to use GenerationConfig or similar for generation_args
training_args.generation_config = transformers.GenerationConfig(
    **vars(generation_args)
)

# Combine arguments into a single Namespace object (if needed)
args = argparse.Namespace(
    **vars(model_args), **vars(data_args), **vars(training_args),
)

args.cache_dir=cachedir

# Control randomness
random.seed(args.seed)
np.random.seed(args.seed)
torch.manual_seed(args.seed)
torch.cuda.manual_seed_all(args.seed)
torch.backends.cudnn.benchmark = False
torch.backends.cudnn.deterministic = True
set_seed(args.seed)  # transformers seed

In [3]:
if 'lora' in args.sortby:
    args.num_layers = 0
logger = logging.getLogger(__name__)
gpus = torch.cuda.device_count()
mempath = (
    f"/rscratch/tpang/kinshuk/RpMKin/llama_ft/{args.dataset}/"
    + f"{args.sortby}"
)
start_memory = [0] * gpus
end_memory = [0] * gpus
peek_memory = 0

if args.verbose:
        task_info = (
            f"\nSeed: {args.seed}\n"
            + f"Dataset: {args.dataset}\n"
            + f"Sort by: {args.sortby}\n"
            + f"Layers to train: {args.num_layers}\n"
        )
        print(task_info)
else:
    datasets_vb_err()
    transformers_vb_err()
    global _tqdm_active
    _tqdm_active = False


Seed: 7
Dataset: alpaca
Sort by: alpha
Layers to train: 12



In [4]:
def memall(gpus=gpus):
    for i in range(gpus):
        start_memory[i] = torch.cuda.memory_allocated(i)
    return sum(start_memory)

model, tokenizer = get_model(args)

for device in range(gpus):
    reset_peak_memory_stats(device=device)
    reset_max_memory_allocated(device=device)

weight_memory = memall()

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Adding special tokens.
Sorted by  alpha
Training layers: ['model.layers.21.self_attn.v_proj', 'model.layers.22.self_attn.v_proj', 'model.layers.16.self_attn.v_proj', 'model.layers.19.self_attn.v_proj', 'model.layers.18.self_attn.v_proj', 'model.layers.30.self_attn.v_proj', 'model.layers.28.self_attn.v_proj', 'model.layers.17.self_attn.v_proj', 'model.layers.27.self_attn.v_proj', 'model.layers.11.self_attn.v_proj', 'model.layers.23.self_attn.o_proj', 'model.layers.3.mlp.up_proj']
Enabling model.layers.3.mlp.up_proj.weight parameter
Enabling model.layers.11.self_attn.v_proj.weight parameter
Enabling model.layers.16.self_attn.v_proj.weight parameter
Enabling model.layers.17.self_attn.v_proj.weight parameter
Enabling model.layers.18.self_attn.v_proj.weight parameter
Enabling model.layers.19.self_attn.v_proj.weight parameter
Enabling model.layers.21.self_attn.v_proj.weight parameter
Enabling model.layers.22.self_attn.v_proj.weight parameter
Enabling model.layers.23.self_attn.o_proj.weight p

In [5]:
def loss_fn(x, y):
    "A Flat CrossEntropy" 
    return torch.nn.functional.cross_entropy(x.view(-1, x.shape[-1]), y.view(-1))

data_module = make_data_module(tokenizer=tokenizer, args=args) # type: ignore
dataset = {k:v for k,v in data_module.items()}
train_dataloader = DataLoader(
    dataset['train_dataset'], # type: ignore
    batch_size=args.per_device_train_batch_size,
    collate_fn=dataset['data_collator'],
    shuffle=True,
)
train_dataloader.__len__()

Splitting train dataset in train and validation according to `eval_dataset_size`


25489

In [6]:
train_losses = []
val_losses = []
val_accs = []
times = []

model.train()
optimizer = torch.optim.Adam(model.parameters(), lr=args.learning_rate)
optimizer.zero_grad()
optimizer_memory = 0
forward_time = 0
backward_time = 0

for epoch in range(1):
    train_loss = 0
    tr_steps = 0
    tick = 0
    total_time = 0
    for step, batch in enumerate((train_dataloader)):

        tick = time.time()
        optimizer.zero_grad()
        curr = memall()
        batch = {k: v.to(model.device) for k, v in batch.items()}
        input_memory = memall() - curr
        
        curr = memall()
        start = time.time()
        output = model(**batch)
        forward_time += time.time() - start
        activation_memory = memall() - curr
        
        curr = memall()
        start = time.time()
        # loss = loss_fn(out.logits, batch["labels"]) / args.gradient_accumulation_steps
        loss = output.loss
        loss.backward()
        backward_time += time.time() - start
        gradient_memory = memall() - input_memory - weight_memory - optimizer_memory

        curr = memall()
        optimizer.step()
        if step == 0:
             optimizer_memory = memall() - curr
    
        loss = loss.cpu()
        train_loss += loss.item()
        tr_steps += 1
        train_losses.append(train_loss/tr_steps)
        if step % 50 == 0:
            print(f'Step: {step}, Train Loss: {train_loss/tr_steps}')
        torch.cuda.empty_cache()
        total_time += time.time() - tick
        times.append(total_time)
        if step == args.max_steps:
            model.eval()
            break

total_memory = memall()

Step: 0, Train Loss: 1.5841203927993774
Step: 50, Train Loss: 1.3310706405078663
Step: 100, Train Loss: 1.2850244369837318
Step: 150, Train Loss: 1.2148389697864355
Step: 200, Train Loss: 1.2144424544638068
Step: 250, Train Loss: 1.1975869113705548
Step: 300, Train Loss: 1.1830967158772225
Step: 350, Train Loss: 1.1710468609618325
Step: 400, Train Loss: 1.1466584839948692
Step: 450, Train Loss: 1.1490224383424497
Step: 500, Train Loss: 1.142504332278541
Step: 550, Train Loss: 1.1358274322293846
Step: 600, Train Loss: 1.1364754121099752
Step: 650, Train Loss: 1.1316224191560997
Step: 700, Train Loss: 1.1258130778057684
Step: 750, Train Loss: 1.120587999514551
Step: 800, Train Loss: 1.118191792765695
Step: 850, Train Loss: 1.1145173816971228
Step: 900, Train Loss: 1.1096800343153148
Step: 950, Train Loss: 1.1063320579753502
Step: 1000, Train Loss: 1.1058452502153553
Step: 1050, Train Loss: 1.1034407803414834
Step: 1100, Train Loss: 1.10134063752116
Step: 1150, Train Loss: 1.0993011231873

In [7]:
optimizer.zero_grad()
model.eval()
trainer=Seq2SeqTrainer(
                model=model,
                tokenizer=tokenizer,
                args=training_args,
                **{k:v for k,v in data_module.items() if k != 'predict_dataset'},
            )
all_metrics = {"run_name": args.run_name}
if args.do_eval:
    all_metrics = eval_func(args, logger, trainer, all_metrics)
total_memory = memall()
peek_memory = sum([max_memory_allocated(i) for i in range(gpus)])


INFO:__main__:*** Evaluate ***


***** eval metrics *****
  eval_loss               =     1.0835
  eval_runtime            = 0:00:54.07
  eval_samples_per_second =     18.494
  eval_steps_per_second   =      2.312


In [8]:
all_metrics

{'run_name': './output',
 'eval_loss': 1.0834892988204956,
 'eval_runtime': 54.0704,
 'eval_samples_per_second': 18.494,
 'eval_steps_per_second': 2.312}

In [9]:
memory_string = (
        f"{param_count(model)}\n"
        f"Dataset          : {args.dataset}\n"
        f"Method           : {args.sortby}\n"
        f"Layers           : {args.num_layers}\n"
        f"Batch size       : {args.per_device_train_batch_size}\n"
        f"Learning Rate    : {args.learning_rate}\n"
        f"Forward time     : {forward_time/60} min\n"
        f"Backward time    : {backward_time/60} min\n"
        f"Weight memory    : {weight_memory / 1e6} MB\n"
        f"Optimizer memory : {optimizer_memory / 1e6} MB\n"
        f"Activation memory: {activation_memory / 1e6} MB\n"
        f"Gradient memory  : {gradient_memory / 1e6} MB\n"
        f"Input memory     : {input_memory / 1e6} MB\n"
        f"Total memory     : {total_memory / 1e6} MB\n"
        f"Peak memory      : {peek_memory / 1e6} MB\n"
    )
print(memory_string)

Total params: 6738.42M, Trainable: 360.71M
(6738.423808, 360.71424)
Dataset          : alpaca
Method           : alpha
Layers           : 12
Batch size       : 2
Learning Rate    : 2e-06
Forward time     : 1.2104967792828878 min
Backward time    : 3.509366476535797 min
Weight memory    : 27087.929344 MB
Optimizer memory : 2885.71392 MB
Activation memory: 4503.892992 MB
Gradient memory  : 1525.43488 MB
Input memory     : 0.008704 MB
Total memory     : 30056.229888 MB
Peak memory      : 41205.979136 MB



In [11]:
# SAVE TRAINING HISTORY
base = {"train_loss": train_losses,"time": times}
savepath = f"./output/{args.dataset}/lr_{args.learning_rate}/batch_{args.per_device_train_batch_size}/{args.sortby}/layers_{args.num_layers}"
if True:
    Path(savepath).mkdir(parents=True, exist_ok=True)
    np.save(os.path.join(savepath, "finetune.npy"), base) # type: ignore
    with open(os.path.join(savepath, "metrics.json"), "w") as fout:
        fout.write(json.dumps(all_metrics))
    log_info = (
        f"\n\n{args.dataset} "
        + f"Batch Size {args.per_device_train_batch_size} "
        + f"{args.sortby} fine-tuning "
        + f"{args.num_layers} Layers"
    )
    logger = get_logger(savepath, "memlog.log")
    logger.info(log_info)
    logger.info(f"\n{memory_string}\n")
    if (args.do_train or args.do_eval or args.do_predict):
        with open(os.path.join(savepath, "metrics.json"), "w") as fout:
            fout.write(json.dumps(all_metrics))

INFO:root:

alpaca Batch Size 2 alpha fine-tuning 12 Layers


2024-04-08 01:26:42;INFO;

alpaca Batch Size 2 alpha fine-tuning 12 Layers


INFO:root:
(6738.423808, 360.71424)
Dataset          : alpaca
Method           : alpha
Layers           : 12
Batch size       : 2
Learning Rate    : 2e-06
Forward time     : 1.2104967792828878 min
Backward time    : 3.509366476535797 min
Weight memory    : 27087.929344 MB
Optimizer memory : 2885.71392 MB
Activation memory: 4503.892992 MB
Gradient memory  : 1525.43488 MB
Input memory     : 0.008704 MB
Total memory     : 30056.229888 MB
Peak memory      : 41205.979136 MB




2024-04-08 01:26:42;INFO;
(6738.423808, 360.71424)
Dataset          : alpaca
Method           : alpha
Layers           : 12
Batch size       : 2
Learning Rate    : 2e-06
Forward time     : 1.2104967792828878 min
Backward time    : 3.509366476535797 min
Weight memory    : 27087.929344 MB
Optimizer memory : 2885.71392 MB
Activation memory: 4503.892992 MB
Gradient memory  : 1525.43488 MB
Input memory     : 0.008704 MB
Total memory     : 30056.229888 MB
Peak memory      : 41205.979136 MB


